In [ ]:
from google.colab import drive
drive.mount('/content/drive')

PROJECT_PATH = '/content/drive/MyDrive/Colab/ECE570/Project'

Mounted at /content/drive


In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.5 MB 5.1 MB/s 
     |████████████████████████████████| 7.6 MB 38.2 MB/s 
     |████████████████████████████████| 182 kB 39.5 MB/s 


In [ ]:
import torch
device = torch.device('cuda' if torch.cuda.is_available else 'cpu')
print(device)

cuda


# Step 1: Get data from file

In [ ]:
import pandas as pd

data = pd.read_csv(f'{PROJECT_PATH}/imdb.csv')

In [ ]:
import numpy as np

sentences = data.iloc[:,:-1].values.flatten()

y_str = data.iloc[:,-1:].values.flatten()
map_func = np.vectorize(lambda a: 1 if a == 'positive' else 0)
y = map_func(y_str)

# Step 2: Prepare data for input into Bert Model

In [ ]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [ ]:
X = tokenizer(sentences.tolist(), return_tensors='pt', truncation=True, padding=True)

# Step 3: Build datasets

In [ ]:
from torch.utils.data import Dataset

class IMDBDataset(Dataset):
  def __init__(self, ids, token_types, attn_masks, y):
    self.ids = ids
    self.token_types = token_types
    self.attn_masks = attn_masks
    self.y = y
  
  def __len__(self):
    return len(self.y)
    
  def __getitem__(self, idx):
    return self.ids[idx], self.token_types[idx], self.attn_masks[idx], self.y[idx]

In [ ]:
from sklearn.model_selection import train_test_split

ids_train, ids_test, token_types_train, token_types_test, attn_mask_train, attn_mask_test, y_train, y_test = train_test_split(X['input_ids'],
  X['token_type_ids'],
  X['attention_mask'],
  y,
  test_size=0.2, 
  random_state=0)

In [ ]:
dataset_train = IMDBDataset(ids_train, token_types_train, attn_mask_train, y_train)
dataset_test = IMDBDataset(ids_test, token_types_test, attn_mask_test, y_test)

In [ ]:
torch.save(dataset_train, f'{PROJECT_PATH}/dataset/train_set.pt')
torch.save(dataset_test, f'{PROJECT_PATH}/dataset/test_set.pt')